In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast
from itertools import chain

### Загрузка данных

In [ ]:
df = pd.read_csv('ptb/ptbxl_database.csv')
df.head()

In [ ]:
all_diagnosis = df['scp_codes'].apply(lambda x: list(ast.literal_eval(x).items())).tolist()

In [ ]:
sorted(list(set(chain(*all_diagnosis))), key=lambda x: x[0])

In [ ]:
path = 'ptb/records500'
example = wfdb.rdsamp(path + '/00000/00001_hr')

In [ ]:
main_lines = example[0].T[:6]

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(example[0].T[6:])

In [ ]:
np.sum(pca.explained_variance_ratio_)